# A moving surface problem:
In this example we consider a moving surface problem:

$$
\dot{u} + ({div}_\Gamma \mathbf{w} ) u - \nu \Delta_\Gamma u = 0 \quad on \quad \Gamma(t),   \qquad  t \in [0,T]   
$$
where $\Gamma(t)$ changes in time (while $\Gamma(t)$ is always smooth) and is described by a level set function $\phi(t)$.

We follow an approach from [1] where we extend the solution u into a small neighborhood $\mathcal{O}(\Gamma(t))$ and there solve the related problem
\begin{align}
  \partial_t u + \mathbf{w} \cdot \nabla u + ({div}_\Gamma \mathbf{w} ) u - \nu \Delta_\Gamma u &= 0 \quad \text{on} \quad \Gamma(t) \\
  \nabla u \cdot \nabla \phi & = 0 \quad \text{in} \quad \mathcal{O}(\Gamma(t)).
\end{align}
The second equation determines u to be a constant extension in normal direction of the solution on the surface. This extension allows to evaluate u at an older time instance on the surface which may have moved as long as the neighborhood is sufficiently large. Hence, we require $\Gamma_h^n \subset \mathcal{O}(\Gamma_h^{n-1})$ where $\Gamma_h^k,~k=n-1,n$ are the discrete surface approximations of two subsequent time steps.

This allows to apply a [method of lines](http://www.scholarpedia.org/article/Method_of_lines) approach. Here, we consider the backward Euler method:

\begin{align}
  \frac{u^n - u^{n-1}}{\Delta t} + \mathbf{w}^n \cdot \nabla u^n + ({div}_\Gamma \mathbf{w}^n ) u^n - \nu \Delta_\Gamma u^n &= 0 \quad \text{on} \quad \Gamma^n \\
  \nabla u^n \cdot \nabla \phi^n & = 0 \quad \text{in} \quad \mathcal{O}(\Gamma^n).
\end{align}

Note that $u^{n-1}$ is well-defined on $\Gamma^n$ as $\Gamma^n \subset \mathcal{O}(\Gamma^{n-1})$.

The realization of the extension is included in each time step solves as a stabilization. This simplifies the realization of the extension, but also allows to obtain reasonable condition number bounds. Let $V_h^n$ be the finite element space to the part of the mesh that is touched by $\mathcal{0}(\Gamma^n)$, then every time step is discretized with (for $u^{n-1} \in V_h^{n-1} \subset L^2(\mathcal{O}(\Gamma^{n-1})$):

Find $u_h^n \in V_h^n$, s.t.
\begin{align}
  \int_{\Gamma_h^n} \left\{ \frac{u_h^n -u_h^{n-1}}{\Delta t} v_h + \frac12 (w_T^e \cdot \nabla_{\Gamma_h} u_h^n v_h - \mathbf{w}_T^e \cdot \nabla_{\Gamma_h} v_h u_h^n) + ({div}_{\Gamma_h} (\mathbf{w}^e -\frac12 \mathbf{w}_T^e) u_h^n v_h \right \} ds_h &  \\
  + \nu \int_{\Gamma_h^n} \nabla_{\Gamma_h} u_h^n \cdot \nabla_{\Gamma_h} v_h ds_h + \rho_n \int_{\mathcal{O}(\Gamma_h^n)} (\mathbf{n}_h \cdot \nabla u_h^n) (\mathbf{n}_h \cdot \nabla v_h^n) d \mathbf{x} & = 0
\end{align}
for all $v_h \in V_h^n$.
Here $\mathbf{w}^e$ and $\mathbf{w}_T^e$ denote the extensions of $\mathbf{w}$ and $\mathbf{w}_T$ to a neighborhood of $\Gamma(t)$.
Literature:
----------------------
[1]: C. Lehrenfeld, M.A. Olshanskii, and X. Xu. A stabilized trace finite element method for partial differential equations on evolving surfaces. arXiv preprint arXiv:1709.07117, 2017. [ [http](https://arxiv.org/abs/1709.07117) | [.pdf](https://arxiv.org/pdf/1709.05832.pdf) ]


In [ ]:
import netgen.gui
%gui tk
import tkinter

from ngsolve import *
from ngsolve.internal import *
from xfem import *
from netgen.geom2d import SplineGeometry


## Description of the time-dependent geometry

We use a levelset function $\phi(t) =  \Vert \mathbf{x}-\mathbf{w}t \Vert - r_{0}$ with
$r_{0} = 1/2$ and $\mathbf{w} = (1,0)$, to describe the evolving geometry. 
The moving surface is then given as the set of points where the levelset function takes zero values:
$ \Gamma(t) = \{ (x,y) \in \Omega \mid \phi(x,y,t) = 0 \}$.
The time-dependent surface $\Gamma(t)$ is contained in a larger, time-independent domain $\Omega = [-1,1.2] \times [-1,1]$.
The mesh is constructed on the background domain $\Omega$ and unfitted to $\Gamma(t)$.
We provide $\mathbf{w}$, $\phi$, $div_{\Gamma} \mathbf{w}^e$ and $div_{\Gamma} \mathbf{w}_T^e$ as functions:

In [ ]:
geom = SplineGeometry()
geom.AddRectangle([-1,-1],[1.2,1],bc=1)
ngmesh = geom.GenerateMesh(maxh=0.125, quad=False)
mesh = Mesh (ngmesh)

vel = 1
R = 0.5
def velocity_function(t):
    return CoefficientFunction((vel,0))
def levelset_function(t):
    return sqrt((x-vel*t)*(x-vel*t)+y*y) - R
def divGwT_function(t):
    return -vel/(R*R)*(x-vel*t)
def divGw_function(t):
    return 0.0


## An example solution
In this geometrical configuration, we consider the following solution (for the initial data):

In [ ]:
def solution_function(t):
    return 1 + (x+y - vel*t)*exp(-2*t)

## P1 level set approximation
The level set function is approximated by a piecewise linear function and corresponding normal and tangential projections are defined.

In [ ]:
t = Parameter(0)

levelset = levelset_function(t)
Draw(levelset,mesh,"lset")

lset_approx = GridFunction(H1(mesh,order=1))
lset_if  = { "levelset" : lset_approx, "domain_type" : IF , "subdivlvl" : 0}
n = 1.0/sqrt(InnerProduct(grad(lset_approx),grad(lset_approx))) * grad(lset_approx)
h = specialcf.mesh_size
  
def P(u):
    return u - (u*n)*n

## P1 TraceFEM
For the TraceFEM we consider the restriction of a standard P1 finite element space to the discrete surface:

In [ ]:
VhG = H1(mesh, order=1, dirichlet=[])
u = VhG.TrialFunction()
v = VhG.TestFunction()
  
gfu = GridFunction(VhG)
gfu.Set(solution_function(0))

## Visualization band of elements
The active band of elements does not only include the elements that are intersected by the discrete interface but also include a band of a size that depends on the maximum velocity and the time step. Quantities are visualized on this band the BitArrays to which are corresponding initialized:

In [ ]:
#visualization of active band elements
visband = GridFunction(L2(mesh,order=0))
nan=float("nan")

Draw(IfPos(visband-0.1,levelset,nan),mesh,"lset")
  
ba_band_IF = BitArray(mesh.ne)
ba_band_IF_P = BitArray(mesh.ne)
ba_band_IF_M = BitArray(mesh.ne)
  
visoptions.mminval = -0.001
visoptions.mmaxval = 0.001
visoptions.deformation = 1
visoptions.scaledeform1 = 0.0001
visoptions.autoscale = 0

def UpdateBand(lset,offset,ba_band_IF):
   ba_band_IF.Clear()
   ba_band_IF_P = BitArray(ba_band_IF)
   ba_band_IF_P.Clear()
   ba_band_IF_M = BitArray(ba_band_IF)
   ba_band_IF_M.Clear()

   # all elements that have level set values in [-offset,+offset] with offset = vel * dt
   lset_approx_band = GridFunction(H1(mesh,order=1))
   InterpolateToP1(lset+offset,lset_approx_band)
   ci = CutInfo(mesh, lset_approx_band)
   ba_band_IF_P |= ci.GetElementsOfType(POS)
   ba_band_IF_P |= ci.GetElementsOfType(IF)
   
   InterpolateToP1(lset-offset,lset_approx_band)
   ci = CutInfo(mesh, lset_approx_band)
   ba_band_IF_M |= ci.GetElementsOfType(NEG)
   ba_band_IF_M |= ci.GetElementsOfType(IF)

   ba_band_IF |= ba_band_IF_P
   ba_band_IF &= ba_band_IF_M
      
   for i in range(mesh.ne):
      visband.vec[i] = ba_band_IF[i]
UpdateBand(levelset_function(0),0.1,ba_band_IF)        
Redraw()

Some parameters:

In [ ]:
told = 0
dt = 0.0025
T = 0.2/vel
nu = 2*R*R

timestep = 0

gfu.Set(solution_function(0))
Draw(IfPos(visband-0.1,gfu,nan),mesh,"gfu")

visoptions.autoscale = 0
visoptions.mminval = 0.5
visoptions.mmaxval = 1.5
visoptions.deformation = 1
visoptions.scaledeform1 = 1


The time stepping loop:
 * determining the new interface position for the new integrals
 * Setting up the band of relevant elements
 * marking dofs accordingly
 * setting up matrix and r.h.s. 
 * solving the linear system
 * continuing ot next time step

In [ ]:
while (told < T-dt/2):
   tnew = told + dt
   t.Set(tnew)

   InterpolateToP1(levelset_function(tnew),lset_approx)
   ci = CutInfo(mesh, lset_approx)
   ba_new_IF = ci.GetElementsOfType(IF)
   
   UpdateBand(levelset_function(tnew),vel*dt,ba_band_IF)

   InterpolateToP1(levelset_function(tnew),lset_approx)
   
   active_dofs = BitArray(VhG.ndof)
   active_dofs.Clear()
   active_dofs |= GetDofsOfElements(VhG,ba_band_IF)

   # a = BilinearForm(VhG, symmetric = False, flags = { })
   a = RestrictedBilinearForm(VhG, element_restriction=ba_band_IF, facet_restriction=None, check_unused=False)

   w = velocity_function(tnew)
   divGw = divGw_function(tnew)
   divGwT = divGwT_function(tnew)
       
   a += SymbolicBFI(levelset_domain = lset_if ,
                         form =
                         dt * (divGw-0.5*divGwT)*u*v
                         + dt * nu * P(grad(u)) * P(grad(v))
                         + dt * 0.5*P(w)*P(grad(u))*v - dt * 0.5*P(w)*P(grad(v))*u
                         + u * v, definedonelements=ba_new_IF)
   rho = (vel + nu/h)
   a += SymbolicBFI(form = dt*rho*(grad(u)*n)*(grad(v)*n), definedonelements=ba_band_IF)

   a.Assemble()
     
   f = LinearForm(VhG)
   f += SymbolicLFI(levelset_domain = lset_if, form = gfu * v,definedonelements=ba_new_IF)
   f.Assemble();

   gfu.vec.data = a.mat.Inverse(active_dofs, inverse="umfpack") * f.vec
   Redraw(blocking=True)

   told = tnew
   timestep += 1

print("")
print("finished run")
print("")
